## Bikes Price Prediction

In [1]:
#importing necessary Libraries
import pandas as pd
import numpy as np
import re
import datetime
import unicodedata

df = pd.read_csv("bikes_data.csv")
df.head()

,Company,Country of Origin,Model,Number of cc,Horsepower,Torque,Transmission Type,Drivetrain,Number of Seating,Price (in INR),Year,Looks,Body Type,Engine Type,Number of Cylinders
0,Aprilia,Italy,RS 660,659,100 hp,67 Nm,6-speed quickshifter,Chain,2,"INR 10,99,000",2021,Sport,Naked,Parallel-twin,2
1,Aprilia,Italy,Tuono 660,659,100 hp,67 Nm,6-speed quickshifter,Chain,2,"INR 11,99,000",2021,Sport,Naked,Parallel-twin,2
2,Aprilia,Italy,RS 125,124.9,15 hp,12 Nm,6-speed manual,Chain,2,"INR 4,49,000",2022,Sport,Racing,Single-cylinder,1
3,Aprilia,Italy,Shiver 900,896,95 hp,90 Nm,6-speed manual,Shaft,2,"INR 13,99,000",2022,Adventure,Naked,V-twin,2
4,Aprilia,Italy,Tuono 1100,1077,175 hp,121 Nm,6-speed manual,Shaft,2,"INR 19,99,000",2022,Adventure,Naked,V-twin,2


First we will take a look at numerical variables

Here in the above dataset numerical variables are Number of cc, Horsepower, Torque, Number of seating, Year, Number of cylinder.
However some variables are by default object dtype as the also have their unit along with them so we will remove unit.

In [2]:
df["Number of cc"].unique()

array(['659', '124.9', '896', '1077', '125', '250', '300', '600', '899',
       '500', '313', '853', '1,170', '999', '149', '492', '984', '1125',
       '1201', '893', '998', '1078', '650', '450', '900', '1200',
       '292.4 cc', '649.3 cc', '649 cc', '1,967 cc', '2,163 cc',
       '124.9 cc', '249.6 cc', '1103', '1262', '803', '937', '1158',
       '975', '1252', '1868', '348.36 cc', '373.2 cc', '693 cc',
       '1133 cc', '1799 cc', '1203 cc', '296', '399', '649', '948', '200',
       '390', '373', '799', '550', '124.7', '295', '199.6', '110',
       '106.7', '1400', '1700', '798', '800', '961', '160', '150', '660',
       '346', '499', '349', '411', '648', '249', '155', '124', '400',
       '2,500', '1,160', '888', '1,200', '70', '889', '321', '350', '693',
       '1133', '248.76', '373.2', '501', '1,301', '1340', '2458', '149.5',
       '199.5', '373.3', '220.9', '100', '293', '334', '159.7', '124.8',
       '750', '29', '125cc', '250cc', '300cc', '106', '1179', '1731',
       '29

The Number of cc Variables tells us about engine displacment, however we can see that above values also contains value such as electric and some values with unit as Kw which indicates that dataset also has some data about electric bikes, so Now we will create 3 new columns and delete existing column "Number of cc" 

* Electric Vehicle : This will contain 1 if it is an ev or else it will contain 0
* Engine Displacement : This column will tell us about Engine Displacment and will be set to 0 for electric vehicles
* Motor Power : This column will tell us about power of electric motor for ice vechicles this will be 0

In [3]:
def isElectric(value):
    if value == "Electric":
        return 1
    value = value.lower()
    value.replace(" ","")
    repattern = re.compile("([0-9.]+)([a-z]*)")
    result = repattern.match(value).groups()
    if result[1] == "":
        return 0
    if result[1] == "kw":
        return 1
    return 0

def getEngineDisplacement(value):
    if value == "Electric":
        return 0
    value = value.lower()
    value.replace(" ","")
    repattern = re.compile("([0-9.]+)([a-z]*)")
    result = repattern.match(value).groups()
    if result[1] == "kw":
        return 0
    return result[0]

def getMotorPower(value):
    if value == "Electric":
        return np.nan
    value = value.lower()
    value.replace(" ","")
    repattern = re.compile("([0-9.]+)([a-z]*)")
    result = repattern.match(value).groups()
    if result[1] == "kw":
        return result[0]
    return 0
    

df["Electric Vehicle"] = df["Number of cc"].apply(isElectric)
df["Engine Displacement"] = pd.to_numeric(df["Number of cc"].apply(getEngineDisplacement))
df["Motor Power"] = pd.to_numeric(df["Number of cc"].apply(getMotorPower))
df.drop("Number of cc",axis=1,inplace=True)
df.head()

,Company,Country of Origin,Model,Horsepower,Torque,Transmission Type,Drivetrain,Number of Seating,Price (in INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,RS 660,100 hp,67 Nm,6-speed quickshifter,Chain,2,"INR 10,99,000",2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
1,Aprilia,Italy,Tuono 660,100 hp,67 Nm,6-speed quickshifter,Chain,2,"INR 11,99,000",2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
2,Aprilia,Italy,RS 125,15 hp,12 Nm,6-speed manual,Chain,2,"INR 4,49,000",2022,Sport,Racing,Single-cylinder,1,0,124.9,0.0
3,Aprilia,Italy,Shiver 900,95 hp,90 Nm,6-speed manual,Shaft,2,"INR 13,99,000",2022,Adventure,Naked,V-twin,2,0,896.0,0.0
4,Aprilia,Italy,Tuono 1100,175 hp,121 Nm,6-speed manual,Shaft,2,"INR 19,99,000",2022,Adventure,Naked,V-twin,2,0,1077.0,0.0


In [4]:
df["Horsepower"].unique()

array(['100 hp', '15 hp', '95 hp', '175 hp', '11 hp', '27 hp', '39 hp',
       '85 hp', '116 hp', '47 hp', '34 hp', '136 hp', '199 hp', '165 hp',
       '11.7 hp', '41 hp', '45 hp', '110 hp', '96 hp', '105 hp', '80 hp',
       '65 hp', '60 hp', '86 hp', '115 hp', '33.99 PS', '61.18 PS',
       '70.70 PS', '62.54 PS', '205 hp', '190 hp', '227 hp', '10.8 hp',
       '11.2 hp', '27.6 hp', '221 hp', '159.6 hp', '73 hp', '111 hp',
       '168 hp', '88.5', '120.7', '92.5', '150.2', '94',
       '26.5 hp @ 8,500 rpm', '31 hp @ 8,500 rpm', '29.6 hp @ 5,500 rpm',
       '10.5 hp @ 7,500 rpm', '8.75 hp @ 7,500 rpm', '43 hp @ 9,000 rpm',
       '75 hp @ 8,500 rpm', '108 hp @ 5,600 rpm', '122 hp @ 6,000 rpm',
       '120 hp @ 8,250 rpm', '94 hp @ 6,000 rpm', '39 PS @ 11000 rpm',
       '45 PS @ 10,000 rpm', '67 PS @ 8000 rpm', '125 PS @ 9500 rpm',
       '197 PS @ 11,000 rpm', '60 PS @ 6500 rpm', '25 hp', '30 hp',
       '43 hp', '10 hp', '29 hp', '12.50 HP @ 10000 RPM',
       '25.8 HP @ 9000 RPM

From the above data we can see that there are a lot of unique values for horsepower with different units we need to have same unit for all the cells.We will convert all value to hp unit

In [5]:
df["Horsepower"] = df["Horsepower"].str.replace(" ","")
df["Horsepower"] = df["Horsepower"].apply(lambda x:x.split("@")[0])
df["Horsepower"] = df["Horsepower"].str.strip()
df["Horsepower"] = df["Horsepower"].str.lower()
df["Horsepower"].unique()

array(['100hp', '15hp', '95hp', '175hp', '11hp', '27hp', '39hp', '85hp',
       '116hp', '47hp', '34hp', '136hp', '199hp', '165hp', '11.7hp',
       '41hp', '45hp', '110hp', '96hp', '105hp', '80hp', '65hp', '60hp',
       '86hp', '115hp', '33.99ps', '61.18ps', '70.70ps', '62.54ps',
       '205hp', '190hp', '227hp', '10.8hp', '11.2hp', '27.6hp', '221hp',
       '159.6hp', '73hp', '111hp', '168hp', '88.5', '120.7', '92.5',
       '150.2', '94', '26.5hp', '31hp', '29.6hp', '10.5hp', '8.75hp',
       '43hp', '75hp', '108hp', '122hp', '120hp', '94hp', '39ps', '45ps',
       '67ps', '125ps', '197ps', '60ps', '25hp', '30hp', '10hp', '29hp',
       '12.50hp', '25.8hp', '38hp', '10.19ps', '12.9ps', '9.4hp', '12hp',
       '18.5hp', '22.5hp', '26.5bhp', '18.1bhp', '10.7bhp', '8.5bhp',
       '8.5ps', '80', '65', '75', '96', '115', '147hp', '208hp', '140hp',
       '105', '177', '11.3', '17.7', '95', '9.5', '23.8', '19.8', '27.2',
       '20.2bhp', '24.5bhp', '47.6bhp', '47ps', '26.5ps', '14.8ps'

In [6]:
def getHorsepower(value):
    repattern = re.compile("([0-9.]+)([a-z]*)")
    number,unit = repattern.match(value).groups()
    number = float(number)
    if unit == "" or unit == "hp":
        return number
    elif unit == "bhp" or unit == "ps":
        return number*0.9863
    elif unit == "kw":
        return number*1.34102
    elif unit == "w":
        return number * 0.00134102
    
df["Horsepower"] = pd.to_numeric(df["Horsepower"].apply(getHorsepower))
df.rename(columns={"Horsepower":"Horsepower (hp)"},inplace=True)
df.head()

,Company,Country of Origin,Model,Horsepower (hp),Torque,Transmission Type,Drivetrain,Number of Seating,Price (in INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,RS 660,100.0,67 Nm,6-speed quickshifter,Chain,2,"INR 10,99,000",2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
1,Aprilia,Italy,Tuono 660,100.0,67 Nm,6-speed quickshifter,Chain,2,"INR 11,99,000",2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
2,Aprilia,Italy,RS 125,15.0,12 Nm,6-speed manual,Chain,2,"INR 4,49,000",2022,Sport,Racing,Single-cylinder,1,0,124.9,0.0
3,Aprilia,Italy,Shiver 900,95.0,90 Nm,6-speed manual,Shaft,2,"INR 13,99,000",2022,Adventure,Naked,V-twin,2,0,896.0,0.0
4,Aprilia,Italy,Tuono 1100,175.0,121 Nm,6-speed manual,Shaft,2,"INR 19,99,000",2022,Adventure,Naked,V-twin,2,0,1077.0,0.0


Now we have converted all the horsepower in same units, we shall apply same procedure for Torque variable

In [7]:
df["Torque"].unique()

array(['67 Nm', '12 Nm', '90 Nm', '121 Nm', '10.8 Nm', '21.2 Nm',
       '26.5 Nm', '54.6 Nm', '86.3 Nm', '46 Nm', '28 Nm', '92 Nm',
       '112 Nm', '113 Nm', '11.9 Nm', '38 Nm', '37 Nm', '97 Nm', '98 Nm',
       '103 Nm', '66 Nm', '95 Nm', '54 Nm', '40 Nm', '77 Nm', '105 Nm',
       '21.5 Nm', '45 Nm', '55 Nm', '49 Nm', '135 lb-ft', '125 lb-ft',
       '143 lb-ft', '10.2 Nm', '22.4 Nm', '124 lb-ft', '93 lb-ft',
       '47 lb-ft', '69 lb-ft', '65 ft-lbs', '94 ft-lbs', '118 ft-lbs',
       '112 ft-lbs', '19.4 Nm @ 7,000 rpm', '27.3 Nm @ 6,500 rpm',
       '30.0 Nm @ 3,000 rpm', '10.3 Nm @ 5,500 rpm', '37 Nm @ 7,000 rpm',
       '72 Nm @ 6,750 rpm', '114 Nm @ 3,200 rpm', '178 Nm @ 3,800 rpm',
       '110 Nm @ 6,000 rpm', '95 Nm @ 3,750 rpm', '27 Nm @ 10000 rpm',
       '39 Nm @ 8000 rpm', '66 Nm @ 6500 rpm', '98.6 Nm @ 7700 rpm',
       '140 Nm @ 8500 rpm', '62 Nm @ 6500 rpm', '19 Nm', '24 Nm', '88 Nm',
       '9.5 Nm', '27 Nm', '10.2 Nm @ 8000 RPM', '23.5 Nm @ 7000 RPM',
       '38 Nm 

In [8]:
df["Torque"] = df["Torque"].apply(lambda x:x.split("@")[0])
df["Torque"] = df["Torque"].str.strip()
df["Torque"] = df["Torque"].str.lower()
df["Torque"] = df["Torque"].str.replace(" ","")
df["Torque"].unique()

array(['67nm', '12nm', '90nm', '121nm', '10.8nm', '21.2nm', '26.5nm',
       '54.6nm', '86.3nm', '46nm', '28nm', '92nm', '112nm', '113nm',
       '11.9nm', '38nm', '37nm', '97nm', '98nm', '103nm', '66nm', '95nm',
       '54nm', '40nm', '77nm', '105nm', '21.5nm', '45nm', '55nm', '49nm',
       '135lb-ft', '125lb-ft', '143lb-ft', '10.2nm', '22.4nm', '124lb-ft',
       '93lb-ft', '47lb-ft', '69lb-ft', '65ft-lbs', '94ft-lbs',
       '118ft-lbs', '112ft-lbs', '19.4nm', '27.3nm', '30.0nm', '10.3nm',
       '72nm', '114nm', '178nm', '110nm', '27nm', '39nm', '98.6nm',
       '140nm', '62nm', '19nm', '24nm', '88nm', '9.5nm', '23.5nm', '50nm',
       '9.2nm', '16nm', '14nm', '18nm', '21nm', '17.3nm', '10.5nm',
       '8.5nm', '73.8lb-ft', '59.4lb-ft', '72lb-ft', '86lb-ft', '89lb-ft',
       '116nm', '86nm', '67', '90', '125', '11.5', '14.2', '9.8', '26',
       '41nm', '32nm', '52nm', '52.3nm', '22.6nm', '26nm', '34nm', '10nm',
       '9nm', '163lb-ft', '95lb-ft', '59lb-ft', '64lb-ft', '62lb-ft'

Torques has different unit such as nm , lb-ft etc we will convert it into nm

In [9]:
def getTorque(value):
    if value == "-":
        return np.nan
    repattern = re.compile("([0-9.]+)([a-z\-]*)")
    number,unit = repattern.match(value).groups()
    number = float(number)
    if unit == "" or unit == "nm":
        return number
    elif unit in ["lb-ft","ft-lbs","ft-lb"]:
        return number*1.36
    
df["Torque"] = pd.to_numeric(df["Torque"].apply(getTorque))
df.rename(columns={"Torque":"Torque (nm)"},inplace=True)
df.head()

,Company,Country of Origin,Model,Horsepower (hp),Torque (nm),Transmission Type,Drivetrain,Number of Seating,Price (in INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,RS 660,100.0,67.0,6-speed quickshifter,Chain,2,"INR 10,99,000",2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
1,Aprilia,Italy,Tuono 660,100.0,67.0,6-speed quickshifter,Chain,2,"INR 11,99,000",2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
2,Aprilia,Italy,RS 125,15.0,12.0,6-speed manual,Chain,2,"INR 4,49,000",2022,Sport,Racing,Single-cylinder,1,0,124.9,0.0
3,Aprilia,Italy,Shiver 900,95.0,90.0,6-speed manual,Shaft,2,"INR 13,99,000",2022,Adventure,Naked,V-twin,2,0,896.0,0.0
4,Aprilia,Italy,Tuono 1100,175.0,121.0,6-speed manual,Shaft,2,"INR 19,99,000",2022,Adventure,Naked,V-twin,2,0,1077.0,0.0


Now we have converted the torque variable with proper units,now lets take a look at Number of seating column

In [10]:
df["Number of Seating"].unique()

array([2, 1, 3], dtype=int64)

Number of seating requires no processing

Now we will see the Price Column, The price is a numerical feature however the cell in price column are along with the units we will remove this units

In [11]:
df["Price (in INR)"].unique()

array(['INR 10,99,000', 'INR 11,99,000', 'INR 4,49,000', 'INR 13,99,000',
       'INR 19,99,000', '1,25,000', '1,75,000', '2,00,000', '4,00,000',
       '6,00,000', '2,25,000', '3,50,000', '3,25,000', '3,00,000',
       '2,50,000', '8,00,000', '12,00,000', '14,00,000', '16,00,000',
       '1.50 lakhs', '1.25 lakhs', '3.50 lakhs', '3.75 lakhs', '4 lakhs',
       '4.5 lakhs', '10 lakhs', '13 lakhs', '11 lakhs', '12 lakhs',
       '1.5 lakhs', '8 lakhs', '15 lakhs', '20 lakhs', 'Rs. 2,29,000',
       'Rs. 4,29,000', 'Rs. 5,29,000', 'Rs. 5,59,000', '$55,000',
       '$50,000', '$100,000', 'Rs. 1,29,500', 'Rs. 1,40,000',
       'Rs. 2,39,000', 'Rs. 2,59,000', '27.41 Lakh', '25.91 Lakh',
       '9.39 Lakh', '12.95 Lakh', '21.48 Lakh', '17,49,000', '18,79,000',
       '40,49,000', '37,49,000', '24,49,000', '20,49,000', '26,59,000',
       '1,45,000', '1,85,000', '1,65,000', '65,000', '62,000', '3,35,000',
       '6,50,000', '7,50,000', '22,999', '23,999', '25,999', '19,999',
       '14,999', 

Lets first fix the entries with dollar sign

In [12]:
df[df["Price (in INR)"].str.startswith("$")]

,Company,Country of Origin,Model,Horsepower (hp),Torque (nm),Transmission Type,Drivetrain,Number of Seating,Price (in INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
44,Confederate,USA,Hellcat,205.0,183.60,6-speed manual,Belt drive,1,"$55,000",2022,Retro,Cruiser,V-twin,2,0,1.0,0.0
45,Confederate,USA,Wraith,190.0,170.00,6-speed manual,Belt drive,1,"$50,000",2022,Modern,Roadster,V-twin,2,0,1.0,0.0
46,Confederate,USA,X132 Hellcat Combat Fighter,227.0,194.48,6-speed manual,Belt drive,1,"$100,000",2023,Modern,Fighter,V-twin,2,0,2.0,0.0


In [13]:
# Dollar Price while cleaning this dataset 1$ = 82.5Rs
df.loc[44,"Price (in INR)"] = str(55000 * 82.5)
df.loc[45,"Price (in INR)"] = str(50000 * 82.5)
df.loc[46,"Price (in INR)"] = str(100000 * 82.5)

Dollar is converted into rs now lets fix remaining entries

In [14]:
df["Price (in INR)"].unique()

array(['INR 10,99,000', 'INR 11,99,000', 'INR 4,49,000', 'INR 13,99,000',
       'INR 19,99,000', '1,25,000', '1,75,000', '2,00,000', '4,00,000',
       '6,00,000', '2,25,000', '3,50,000', '3,25,000', '3,00,000',
       '2,50,000', '8,00,000', '12,00,000', '14,00,000', '16,00,000',
       '1.50 lakhs', '1.25 lakhs', '3.50 lakhs', '3.75 lakhs', '4 lakhs',
       '4.5 lakhs', '10 lakhs', '13 lakhs', '11 lakhs', '12 lakhs',
       '1.5 lakhs', '8 lakhs', '15 lakhs', '20 lakhs', 'Rs. 2,29,000',
       'Rs. 4,29,000', 'Rs. 5,29,000', 'Rs. 5,59,000', '4537500.0',
       '4125000.0', '8250000.0', 'Rs. 1,29,500', 'Rs. 1,40,000',
       'Rs. 2,39,000', 'Rs. 2,59,000', '27.41 Lakh', '25.91 Lakh',
       '9.39 Lakh', '12.95 Lakh', '21.48 Lakh', '17,49,000', '18,79,000',
       '40,49,000', '37,49,000', '24,49,000', '20,49,000', '26,59,000',
       '1,45,000', '1,85,000', '1,65,000', '65,000', '62,000', '3,35,000',
       '6,50,000', '7,50,000', '22,999', '23,999', '25,999', '19,999',
       '14,9

In [15]:
df["Price (in INR)"] = df["Price (in INR)"].str.lower()
df["Price (in INR)"] = df["Price (in INR)"].str.replace(",","")
df["Price (in INR)"] = df["Price (in INR)"].str.replace(" ","")
df["Price (in INR)"] = df["Price (in INR)"].str.replace("₹","")
df["Price (in INR)"] = df["Price (in INR)"].str.replace("rs.","")
df["Price (in INR)"] = df["Price (in INR)"].str.replace("inr","")
df["Price (in INR)"] = df["Price (in INR)"].str.replace("\(ex-showroom\)","")
df["Price (in INR)"] = df["Price (in INR)"].str.replace("\(expected\)","")
df["Price (in INR)"].unique()

C:\Users\mohit\AppData\Local\Temp\ipykernel_19616\1188776364.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Price (in INR)"] = df["Price (in INR)"].str.replace("rs.","")
C:\Users\mohit\AppData\Local\Temp\ipykernel_19616\1188776364.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Price (in INR)"] = df["Price (in INR)"].str.replace("\(ex-showroom\)","")
C:\Users\mohit\AppData\Local\Temp\ipykernel_19616\1188776364.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Price (in INR)"] = df["Price (in INR)"].str.replace("\(expected\)","")


array(['1099000', '1199000', '449000', '1399000', '1999000', '125000',
       '175000', '200000', '400000', '600000', '225000', '350000',
       '325000', '300000', '250000', '800000', '1200000', '1400000',
       '1600000', '1.50lakhs', '1.25lakhs', '3.50lakhs', '3.75lakhs',
       '4lakhs', '4.5lakhs', '10lakhs', '13lakhs', '11lakhs', '12lakhs',
       '1.5lakhs', '8lakhs', '15lakhs', '20lakhs', '229000', '429000',
       '529000', '559000', '4537500.0', '4125000.0', '8250000.0',
       '129500', '140000', '239000', '259000', '27.41lakh', '25.91lakh',
       '9.39lakh', '12.95lakh', '21.48lakh', '1749000', '1879000',
       '4049000', '3749000', '2449000', '2049000', '2659000', '145000',
       '185000', '165000', '65000', '62000', '335000', '650000', '750000',
       '22999', '23999', '25999', '19999', '14999', '3.43lakh',
       '5.19lakh', '6.65lakh', '9.11lakh', '23.02lakh', '6.95lakh',
       '1.78lakh', '1.92lakh', '2.20lakh', '3.10lakh', '2.05lakh',
       '3.30lakh', '3.60lak

Here the column has entries in various different formats sometimes a value is given while some data is given as range

For data given in range we will take mean of the range to get a single value

In [16]:
def getPrice(value):
    # For type 1 only number or number followed by lakh eg 300000 or 3 lakh
    repattern = re.compile("([0-9.]+)([a-z]*)")
    result = repattern.match(value)
    if result != None:
        result = result.groups()
        if result[1] == "lakh":
            return float(result[0]) * 100000
        return float(result[0])
    
    # For Type 2 range without lakh eg 5000-6000
    repattern = re.compile("([0-9]+)\-([0-9]+)")
    result = repattern.match(value)
    if result != None:
        result = result.groups()
        return (float(result[0]) + float(result[1]) ) / 2
    
    # For Type 3 range with lakhs eg 1lakh-2lakh
    repattern = re.compile("([0-9.]+)([a-z]+)\-([0-9.]+)([a-z]+)")
    result = repattern.match(value)
    if result != None:
        result = result.groups()
        return ( float(result[0]) + float(result[2]) ) / 2
    
df["Price (in INR)"] = pd.to_numeric(df["Price (in INR)"].apply(getPrice))
df.rename(columns={"Price (in INR)":"Price (INR)"},inplace=True)
df.head()

,Company,Country of Origin,Model,Horsepower (hp),Torque (nm),Transmission Type,Drivetrain,Number of Seating,Price (INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,RS 660,100.0,67.0,6-speed quickshifter,Chain,2,1099000.0,2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
1,Aprilia,Italy,Tuono 660,100.0,67.0,6-speed quickshifter,Chain,2,1199000.0,2021,Sport,Naked,Parallel-twin,2,0,659.0,0.0
2,Aprilia,Italy,RS 125,15.0,12.0,6-speed manual,Chain,2,449000.0,2022,Sport,Racing,Single-cylinder,1,0,124.9,0.0
3,Aprilia,Italy,Shiver 900,95.0,90.0,6-speed manual,Shaft,2,1399000.0,2022,Adventure,Naked,V-twin,2,0,896.0,0.0
4,Aprilia,Italy,Tuono 1100,175.0,121.0,6-speed manual,Shaft,2,1999000.0,2022,Adventure,Naked,V-twin,2,0,1077.0,0.0


Price Column values are fixed,Now lets take a look at year column

In [17]:
print(df["Year"].unique())
print(df["Year"].dtype)

[2021 2022 2023 2005 2003 2006 2002 1994 1998 2000 2015 2019 2017 2014
 2016 2020 2018]
int64


No PreProcessing is required for year column

Now lets take a look at Number of Cylinders

In [18]:
df["Number of Cylinders"].unique()

array(['2', '1', '3', '4', '90°', '0', 'Single', nan, 'One', 'Two',
       'Three', 'Parallel-Twin'], dtype=object)

There is some error in the data it should contain a integer but some entries are 90degree etc some entries are in alphabetic form we will need to change thoose

In [19]:
df["Number of Cylinders"].replace(["Single","One","1","Two","2","Parallel-Twin","Three","3","4","0"],[1,1,1,2,2,2,3,3,4,0],inplace=True)
df["Number of Cylinders"].unique()

array([2, 1, 3, 4, '90°', 0, nan], dtype=object)

Now we have replaced alphabetic values with numeric value still there is some error, by looking at the data we can say that engine type may also tell us about the number of cylinders so to solve the remaining error we may look at engine type

In [20]:
df[df["Number of Cylinders"]=="90°"]

,Company,Country of Origin,Model,Horsepower (hp),Torque (nm),Transmission Type,Drivetrain,Number of Seating,Price (INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
111,Moto Guzzi,Italy,V85 TT,80.0,100.368,6-speed manual,Shaft drive,2,1540000.0,2022,Adventure,Standard,V-twin,90°,0,853.0,0.0
112,Moto Guzzi,Italy,V9 Bobber,65.0,80.784,6-speed manual,Shaft drive,2,1360000.0,2016,Bobber,Cruiser,V-twin,90°,0,900.0,0.0
113,Moto Guzzi,Italy,V9 Roamer,65.0,80.784,6-speed manual,Shaft drive,2,1330000.0,2016,Cruiser,Cruiser,V-twin,90°,0,900.0,0.0
114,Moto Guzzi,Italy,Audace,75.0,97.920,6-speed manual,Shaft drive,2,1480000.0,2019,Cruiser,Cruiser,V-twin,90°,0,900.0,0.0
115,Moto Guzzi,Italy,Eldorado,96.0,116.960,6-speed manual,Shaft drive,2,1750000.0,2016,Classic,Cruiser,V-twin,90°,0,1400.0,0.0
116,Moto Guzzi,Italy,MGX-21,115.0,121.040,6-speed automatic,Shaft drive,2,2490000.0,2021,Cruiser,Cruiser,V-twin,90°,0,1700.0,0.0


We can see that the engine type is V-twin for data where number of cylinder = 90° so we can replace that with 2

In [21]:
df["Number of Cylinders"].replace(["90°"],[2],inplace=True)

In [22]:
df[df["Number of Cylinders"] != df["Number of Cylinders"]]

,Company,Country of Origin,Model,Horsepower (hp),Torque (nm),Transmission Type,Drivetrain,Number of Seating,Price (INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
181,GasGas,Spain,SM 700,74.0,70.0,Six-speed,Chain drive,1,1450000.0,2023,Modern,Supermoto,Single,NaN,0,693.0,0.0


In [23]:
df["Number of Cylinders"].replace([np.nan],[1],inplace=True)
df["Number of Cylinders"] = pd.to_numeric(df["Number of Cylinders"])
df["Number of Cylinders"].unique()

array([2., 1., 3., 4., 0.])

Now we have fixed errors in Number of Cylinders and we have processed all the numerical variables

Now we will process all categorical variables

In [24]:
df.head()

,Company,Country of Origin,Model,Horsepower (hp),Torque (nm),Transmission Type,Drivetrain,Number of Seating,Price (INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,RS 660,100.0,67.0,6-speed quickshifter,Chain,2,1099000.0,2021,Sport,Naked,Parallel-twin,2.0,0,659.0,0.0
1,Aprilia,Italy,Tuono 660,100.0,67.0,6-speed quickshifter,Chain,2,1199000.0,2021,Sport,Naked,Parallel-twin,2.0,0,659.0,0.0
2,Aprilia,Italy,RS 125,15.0,12.0,6-speed manual,Chain,2,449000.0,2022,Sport,Racing,Single-cylinder,1.0,0,124.9,0.0
3,Aprilia,Italy,Shiver 900,95.0,90.0,6-speed manual,Shaft,2,1399000.0,2022,Adventure,Naked,V-twin,2.0,0,896.0,0.0
4,Aprilia,Italy,Tuono 1100,175.0,121.0,6-speed manual,Shaft,2,1999000.0,2022,Adventure,Naked,V-twin,2.0,0,1077.0,0.0


In [25]:
df["Country of Origin"].unique()

array(['Italy', 'Germany', 'Austria', 'United States', 'United Kingdom',
       'China', 'USA', 'South Korea', 'Japan', 'India', 'Sweden',
       'Taiwan', 'UK', 'Spain', 'France', 'Denmark', 'Canada'],
      dtype=object)

Here there is entry for United States as well as USA(also United Kingdom and UK)so will have to fix that

In [26]:
df["Country of Origin"].replace(["USA","UK"],["United States","United Kingdom"],inplace=True)
df["Country of Origin"].unique()

array(['Italy', 'Germany', 'Austria', 'United States', 'United Kingdom',
       'China', 'South Korea', 'Japan', 'India', 'Sweden', 'Taiwan',
       'Spain', 'France', 'Denmark', 'Canada'], dtype=object)

Model Name will have no effect on the price of bike hence we can drop model column

In [27]:
df.drop("Model",axis=1,inplace=True)

In [28]:
df["Transmission Type"].unique()

array(['6-speed quickshifter', '6-speed manual', 'Manual',
       '6-speed Manual', '5-speed manual', 'CVT',
       '6-speed sequential manual', '6-speed automatic', 'Automatic',
       '4-speed manual', '6-speed sequential', 'Six-speed sequential',
       'Five-speed constant mesh', 'Six-speed', 'Direct drive',
       'Constant Mesh, 6-speed', '4-speed constant mesh',
       '5-speed constant mesh', '6-speed constant mesh', '5-speed',
       '4-speed', 'BAFANG M600 mid-drive motor',
       'Bafang BBS02 mid-drive motor', 'BLDC hub motor', 'Single Speed',
       'Six-speed constant mesh', '6 Speed Manual', 'CVT Automatic',
       'CVT automatic', '6-speed', 'Pedal-assist and throttle', '6-Speed'],
      dtype=object)

There are alot of different entries of Transmission type some are for ICE engine while some are for electric motor, also since majorly transmission type depends alot upon engine specification we can drop Transmission type column

In [29]:
df.drop("Transmission Type",axis=1,inplace=True)
df.head()

,Company,Country of Origin,Horsepower (hp),Torque (nm),Drivetrain,Number of Seating,Price (INR),Year,Looks,Body Type,Engine Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,100.0,67.0,Chain,2,1099000.0,2021,Sport,Naked,Parallel-twin,2.0,0,659.0,0.0
1,Aprilia,Italy,100.0,67.0,Chain,2,1199000.0,2021,Sport,Naked,Parallel-twin,2.0,0,659.0,0.0
2,Aprilia,Italy,15.0,12.0,Chain,2,449000.0,2022,Sport,Racing,Single-cylinder,1.0,0,124.9,0.0
3,Aprilia,Italy,95.0,90.0,Shaft,2,1399000.0,2022,Adventure,Naked,V-twin,2.0,0,896.0,0.0
4,Aprilia,Italy,175.0,121.0,Shaft,2,1999000.0,2022,Adventure,Naked,V-twin,2.0,0,1077.0,0.0


Lets see tht drivetrain column

In [30]:
df["Drivetrain"].unique()

array(['Chain', 'Shaft', 'Belt', 'Chain drive', 'Shaft drive',
       'Belt drive', 'Automatic', 'Chain Drive', 'Belt Drive', 'CVT',
       'Electric', 'Permanent magnet synchronous motor',
       'Shimano drivetrain', 'Front Suspension', 'Single-gear',
       'Automatic CVT', 'Dual-motor, all-wheel drive',
       'V-twin, liquid-cooled, 4-stroke', 'Front wheel drive',
       'Rear wheel', 'Parallel twin'], dtype=object)

This column has many entries, some entries are about how the wheel is powered like "Chain","Shaft","Belt" etc while some entries are about how which wheel is powered like "Front wheel drive" while other entries include data about transmission like "CVT" etc as this column does not give any meaningful information we will drop this column

In [31]:
df.drop("Drivetrain",axis=1,inplace=True)

Now we will see the looks columns

In [32]:
df["Looks"].unique()

array(['Sport', 'Adventure', 'Classy', 'Sporty', 'Retro', 'Touring',
       'Classic', 'Modern', 'Bold', 'Sleek', 'Modern, stylish',
       'Modern, rugged', 'Modern, sporty', 'Modern, off-road',
       'Classic, rugged', 'Classic, bobber', 'Modern, muscular',
       'Modern, bobber', 'Aggressive', 'Race-inspired', 'Commuter',
       'Bobber', 'Cruiser', 'Futuristic', 'Practical', 'Classy, cruiser',
       'Sporty, aggressive', 'Adventure, touring', 'Adventure, sporty',
       'Classic, stylish', 'Classic, vintage',
       'Sporty, beginner-friendly', 'Motocross', 'Enduro',
       'Classy, aggressive', 'Bold, aggressive', 'Simple, reliable',
       'Fun', 'Budget', 'Rugged', 'Retro-modern', 'Sharp',
       'Modern, aggressive', 'Retro, classic', 'Naked', 'Dirt', 'Stylish',
       'Urban', 'Retro-inspired'], dtype=object)

It seems no processing is required for looks column so lets see body type

In [33]:
df["Body Type"].unique()

array(['Naked', 'Racing', 'Sport', 'Adventure', 'Cruiser', 'Roadster',
       'Superbike', 'Cafe Racer', 'Bobber', 'Scrambler', 'Sportbike',
       'Adventure touring', 'Supermoto', 'Sport-touring', 'Fighter',
       'Scooter', 'Sports', 'Standard', 'Tourer', 'Enduro', 'Supersport',
       'Sport touring', 'Cafe racer', 'Café racer', 'Sports bike',
       'Adventure bike', 'Naked bike', 'Street bike', 'Maxi-Scooter',
       'Naked Bike', 'Retro', 'Dual-sport', 'Motocross', 'Trail',
       'Naked streetfighter', 'Street', 'Street naked', 'commuter',
       'Mountain bike', 'Hybrid bike', 'Electric scooter',
       'Mountain Bike', 'Street Bike', 'Streetfighter', 'Touring',
       'Sports Naked', 'Commuter', 'Dirt bike', 'Off-road', 'Cargo',
       'Classic', 'Bicycle', 'Trail Bike', 'Adventure tourer',
       'Street scrambler', 'Tracker'], dtype=object)

Here it requires some processing like entries for "Cafe Racer" and "Cafe racer" also we have to remove accents from words

In [34]:
def getBodyType(value):
    # removing accents
#     value = unicode(value,"utf-8")
    value = unicodedata.normalize("NFD",value)
    value = value.encode("ascii","ignore")
    value = value.decode("utf-8")
    value = str(value)
    
    value = value.lower()
    value = value.strip()
    x = value.split(" ")
    if len(x) > 1 and x[1] == "bike":
        return x[0]
    return value

df["Body Type"] = df["Body Type"].apply(getBodyType)
x = df["Body Type"].unique()
x.sort()
print(x)

['adventure' 'adventure tourer' 'adventure touring' 'bicycle' 'bobber'
 'cafe racer' 'cargo' 'classic' 'commuter' 'cruiser' 'dirt' 'dual-sport'
 'electric scooter' 'enduro' 'fighter' 'hybrid' 'maxi-scooter' 'motocross'
 'mountain' 'naked' 'naked streetfighter' 'off-road' 'racing' 'retro'
 'roadster' 'scooter' 'scrambler' 'sport' 'sport touring' 'sport-touring'
 'sportbike' 'sports' 'sports naked' 'standard' 'street' 'street naked'
 'street scrambler' 'streetfighter' 'superbike' 'supermoto' 'supersport'
 'tourer' 'touring' 'tracker' 'trail']


Majority of processing is done but still there are some irregularitest liek "sport","sports" and "sport-touring","sport touring"

In [35]:
df["Body Type"].replace(["sports","sportbike","sport-touring","touring"],["sport","sport","sport touring","tourer"],inplace=True)
x = df["Body Type"].unique()
x.sort()
print(x)

['adventure' 'adventure tourer' 'adventure touring' 'bicycle' 'bobber'
 'cafe racer' 'cargo' 'classic' 'commuter' 'cruiser' 'dirt' 'dual-sport'
 'electric scooter' 'enduro' 'fighter' 'hybrid' 'maxi-scooter' 'motocross'
 'mountain' 'naked' 'naked streetfighter' 'off-road' 'racing' 'retro'
 'roadster' 'scooter' 'scrambler' 'sport' 'sport touring' 'sports naked'
 'standard' 'street' 'street naked' 'street scrambler' 'streetfighter'
 'superbike' 'supermoto' 'supersport' 'tourer' 'tracker' 'trail']


Now the processing for Body Type is done so lets take a look at Engine Type

In [36]:
df["Engine Type"].unique()

array(['Parallel-twin', 'Single-cylinder', 'V-twin', 'Single', 'Triple',
       'Boxer', 'Liquid-cooled inline-4', 'Single-cylinder, air-cooled',
       'Single-cylinder, liquid-cooled', 'V-twin, liquid-cooled',
       'V-twin, air-cooled', 'Parallel-twin, liquid-cooled', 'V4',
       'L-Twin', 'Liquid-cooled, parallel-twin',
       'Liquid-cooled, four-cylinder', 'Supercharged, four-cylinder',
       'Air Cooled, Single Cylinder, 4-Stroke',
       'Liquid Cooled, Single Cylinder, 4-Stroke',
       'Single-cylinder, liquid-cooled, DOHC',
       'Single-cylinder, air-cooled, OHC', '3-cylinder',
       'Air-cooled, parallel-twin', 'Liquid-cooled, V4',
       'Air-cooled, single-cylinder', 'Liquid-cooled, single-cylinder',
       'Parallel twin', 'Liquid-cooled, 4-stroke, single-cylinder',
       'Air-cooled, 4-stroke, single-cylinder', '2-cylinder',
       '1-cylinder', 'Electric', '4-cylinder', 'Two-stroke',
       'Four-stroke', 'In-line-four', 'In-line-three',
       'Liquid-cooled, 4

The Engine type column does not give any meaningful information , some entries are about how pistons are positioned while some entries are about how engine is cooled etc, also as we have columns for engine displacement and number of cylinders is our dataset so we will delete this column

In [37]:
df.drop("Engine Type",axis=1,inplace=True)
df.head()

,Company,Country of Origin,Horsepower (hp),Torque (nm),Number of Seating,Price (INR),Year,Looks,Body Type,Number of Cylinders,Electric Vehicle,Engine Displacement,Motor Power
0,Aprilia,Italy,100.0,67.0,2,1099000.0,2021,Sport,naked,2.0,0,659.0,0.0
1,Aprilia,Italy,100.0,67.0,2,1199000.0,2021,Sport,naked,2.0,0,659.0,0.0
2,Aprilia,Italy,15.0,12.0,2,449000.0,2022,Sport,racing,1.0,0,124.9,0.0
3,Aprilia,Italy,95.0,90.0,2,1399000.0,2022,Adventure,naked,2.0,0,896.0,0.0
4,Aprilia,Italy,175.0,121.0,2,1999000.0,2022,Adventure,naked,2.0,0,1077.0,0.0


Now lets take a look at missing values in our dataset

In [39]:
df.isnull().sum()

Company                0
Country of Origin      0
Horsepower (hp)        0
Torque (nm)            1
Number of Seating      0
Price (INR)            0
Year                   0
Looks                  0
Body Type              0
Number of Cylinders    0
Electric Vehicle       0
Engine Displacement    0
Motor Power            1
dtype: int64

This dataset only has 2 missing values both of which are numerical type so rather than imputing these values it is better that we drop thoose rows

In [41]:
df.dropna(inplace=True)
df.isnull().sum()

Company                0
Country of Origin      0
Horsepower (hp)        0
Torque (nm)            0
Number of Seating      0
Price (INR)            0
Year                   0
Looks                  0
Body Type              0
Number of Cylinders    0
Electric Vehicle       0
Engine Displacement    0
Motor Power            0
dtype: int64

Now there are no missing value and we have processed the data, so we can focus on prepraing the ML Model

In [38]:
df.to_csv("bikes_data_cleaned.csv",index=False)